In [2]:
import pandas as pd
import sqlalchemy as sql

from sqlalchemy import create_engine


In [3]:
#Conexão com BD
DATABASE_URI = 'mysql+mysqlconnector://{user}:{password}@{server}/{database}'.format(
    user='root', 
    password='125478', 
    server='127.0.0.1', 
    database='analise_risco')

cnx = sql.create_engine(DATABASE_URI)

In [19]:
#Infos Relevantes
# 14952 clientes que não tiveram registros na tabela id (analise_risco.id {Tabela Fato})
#

# df = pd.read_sql('select * from analise_risco.dados_mutuarios left join analise_risco.id on(dados_mutuarios.person_id=id.person_id)',
#                  cnx)

df = pd.read_sql('select pessoa_id from analise_risco.dados_mutuarios',cnx)

df.head()

,pessoa_id
0,tad8vkTn-vp4l
1,dEILQqp2-lgLx
2,Az11sWWw-SdFA
3,YDovEwBH-0eek
4,lNsbfwlP-gzOE


In [22]:
duplicadas = df.duplicated()

duplicadas

0        False
1        False
2        False
3        False
4        False
         ...  
34484    False
34485    False
34486    False
34487    False
34488    False
Length: 34489, dtype: bool